In [ ]:
import random
import matplotlib.pyplot as plt
MAX = 51

def bubble_sort_graph(C):
    n = len(C)
    time = 0
    for i in range(n):
        swapped = False
        newLast = n - i - 1
        for j in range(newLast):
            time += 1
            if C[j] > C[j + 1]:
                temp = C[j]
                C[j] = C[j + 1]
                C[j + 1] = temp
                swapped = True
        if not swapped:
            break
    return time

x = []
y_mejor = []
y_peor = []
y_prom = []

for n in range(1, MAX):
    x.append(n)

    C_mejor = []
    C_peor = []
    C_prom = []
    
    for i in range(n):
        C_mejor.append(i)
        C_peor.append(n - i - 1)
        C_prom.append(random.randint(-3000, 3000))
    
    y_mejor.append(bubble_sort_graph(C_mejor))
    y_peor.append(bubble_sort_graph(C_peor))
    y_prom.append(bubble_sort_graph(C_prom))

##################################################################################

def ajuste_lineal(x, y):
    n = len(x)
    sum_x = sum(x)
    sum_y = sum(y)
    sum_x2 = sum(i*i for i in x)
    sum_xy = sum(x[i]*y[i] for i in range(n))

    a = (n*sum_xy - sum_x*sum_y) / (n*sum_x2 - sum_x**2)
    b = (sum_y - a*sum_x) / n
    return a, b

def ajuste_cuadratico(x, y):
    n = len(x)
    Sx  = sum(x)
    Sx2 = sum(i**2 for i in x)
    Sx3 = sum(i**3 for i in x)
    Sx4 = sum(i**4 for i in x)

    Sy   = sum(y)
    Sxy  = sum(x[i]*y[i] for i in range(n))
    Sx2y = sum((x[i]**2)*y[i] for i in range(n))

    # Sistema de ecuaciones 3x3
    A = [[Sx4, Sx3, Sx2],
         [Sx3, Sx2, Sx],
         [Sx2, Sx, n]]
    B = [Sx2y, Sxy, Sy]

    # Eliminación de Gauss
    for i in range(3):
        factor = A[i][i]
        for j in range(3):
            A[i][j] /= factor
        B[i] /= factor

        for k in range(3):
            if k != i:
                factor = A[k][i]
                for j in range(3):
                    A[k][j] -= factor * A[i][j]
                B[k] -= factor * B[i]

    a, b, c = B
    return a, b, c

##################################################################################

a_m, b_m = ajuste_lineal(x, y_mejor)
print(f"Mejor caso ≈ y = {a_m:.2f}x + {b_m:.2f}")

a_p, b_p, c_p = ajuste_cuadratico(x, y_peor)
print(f"Peor caso  ≈ y = {a_p:.2f}x² + {b_p:.2f}x + {c_p:.2f}")

a_pr, b_pr, c_pr = ajuste_cuadratico(x, y_prom)
print(f"Promedio   ≈ y = {a_pr:.2f}x² + {b_pr:.2f}x + {c_pr:.2f}")

##################################################################################

plt.title("Mejor Caso")
plt.plot(x, y_mejor)
plt.grid(True)
plt.show()
plt.title("Peor Caso")
plt.plot(x, y_peor)
plt.grid(True)
plt.show()
plt.title("Caso Promedio")
plt.plot(x, y_prom)
plt.grid(True)
plt.show()

## Merge Sort

In [ ]:
import random
import matplotlib.pyplot as plt
import math
MAX = 51

# Merge Sort con conteo de operaciones
def merge_sort_count(arr):
    if len(arr) <= 1:
        return arr, 0
    
    mid = len(arr) // 2
    L, count_L = merge_sort_count(arr[:mid])
    R, count_R = merge_sort_count(arr[mid:])
    
    merged, count_merge = merge(L, R)
    
    return merged, count_L + count_R + count_merge

def merge(L, R):
    merged = []
    i = j = 0
    count = 0
    
    while i < len(L) and j < len(R):
        count += 1
        if L[i] <= R[j]:
            merged.append(L[i])
            i += 1
        else:
            merged.append(R[j])
            j += 1
    
    merged.extend(L[i:])
    merged.extend(R[j:])
    
    return merged, count

##################################################################################

x = []
y_mejor = []
y_peor = []
y_prom = []

for n in range(1, MAX):
    x.append(n)

    # Mejor caso: ya ordenado
    C_mejor = list(range(n))
    # Peor caso: ordenado inverso
    C_peor = list(range(n, 0, -1))
    # Promedio: aleatorio
    C_prom = [random.randint(-3000, 3000) for _ in range(n)]
    
    _, count_mejor = merge_sort_count(C_mejor)
    _, count_peor = merge_sort_count(C_peor)
    _, count_prom = merge_sort_count(C_prom)
    
    y_mejor.append(count_mejor)
    y_peor.append(count_peor)
    y_prom.append(count_prom)

##################################################################################
# Ajuste aproximado con forma a*n*log(n) + b*n + c usando mínimos cuadrados

def ajuste_nlogn(x, y):
    n = len(x)
    X1 = [xi * math.log(xi, 2) if xi > 0 else 0 for xi in x]  # n*log2(n)
    X2 = x[:]   # n
    X3 = [1]*n  # constante
    
    # Sumas
    S1 = sum(X1); S2 = sum(X2); S3 = n
    S11 = sum(xi*xi for xi in X1); S22 = sum(xi*xi for xi in X2); S33 = n
    S12 = sum(X1[i]*X2[i] for i in range(n))
    S13 = sum(X1)
    S23 = sum(X2)
    Sy  = sum(y)
    S1y = sum(X1[i]*y[i] for i in range(n))
    S2y = sum(X2[i]*y[i] for i in range(n))

    # Sistema de 3x3: A·coef = B
    A = [[S11, S12, S13],
         [S12, S22, S23],
         [S13, S23, S3]]
    B = [S1y, S2y, Sy]

    # Eliminación de Gauss
    for i in range(3):
        factor = A[i][i]
        for j in range(3):
            A[i][j] /= factor
        B[i] /= factor
        for k in range(3):
            if k != i:
                factor = A[k][i]
                for j in range(3):
                    A[k][j] -= factor * A[i][j]
                B[k] -= factor * B[i]

    a, b, c = B
    return a, b, c

##################################################################################
# Mostrar aproximaciones
a_m, b_m, c_m = ajuste_nlogn(x, y_mejor)
print(f"Mejor caso  ≈ y = {a_m:.2f}·n·log₂(n) + {b_m:.2f}·n + {c_m:.2f}")

a_p, b_p, c_p = ajuste_nlogn(x, y_peor)
print(f"Peor caso   ≈ y = {a_p:.2f}·n·log₂(n) + {b_p:.2f}·n + {c_p:.2f}")

a_pr, b_pr, c_pr = ajuste_nlogn(x, y_prom)
print(f"Promedio    ≈ y = {a_pr:.2f}·n·log₂(n) + {b_pr:.2f}·n + {c_pr:.2f}")

##################################################################################
# Graficar
plt.title("Mejor Caso Merge Sort")
plt.plot(x, y_mejor, label="Datos")
plt.grid(True)
plt.legend()
plt.show()

plt.title("Peor Caso Merge Sort")
plt.plot(x, y_peor, label="Datos")
plt.grid(True)
plt.legend()
plt.show()

plt.title("Caso Promedio Merge Sort")
plt.plot(x, y_prom, label="Datos")
plt.grid(True)
plt.legend()
plt.show()


In [ ]:
def merge_sort(B):
    if len(B) <= 1:
        return B

    m = len(B) // 2
    L = merge_sort(B[:m])
    R = merge_sort(B[m:])
    return merge(L, R)

def merge(L, R):
    B_merge = []
    i = 0
    j = 0
    
    while len(L) > i and len(R) > j:
        if L[i] <= R[j]:
            B_merge.append(L[i])
            i = i + 1
        else:
            B_merge.append(R[j])
            j = j + 1

    return B_merge + L[i:] + R[j:]

B = [11, 4, 7, 9, 1, 5, 2]
print(B)
print(merge_sort(B))

In [ ]:
import matplotlib.pyplot as plt
import random

MAX = 100

def merge_sort_graph(B):
    sorted_list, times = merge_sort(B)
    return times


def merge_sort(B):
    if len(B) <= 1:
        return B, 0   # lista y contador

    m = len(B) // 2
    L, L_times = merge_sort(B[:m])
    R, R_times = merge_sort(B[m:])
    B_merge, merge_times = merge(L, R)

    return B_merge, L_times + R_times + merge_times


def merge(L, R):
    B_merge = []
    i = j = times = 0

    while i < len(L) and j < len(R):
        times += 1
        if L[i] <= R[j]:
            B_merge.append(L[i])
            i += 1
        else:
            B_merge.append(R[j])
            j += 1

    return B_merge + L[i:] + R[j:], times

x = []
y_mejor = []
B_mejor = []

for n in range(1, MAX + 1):
    B_mejor.append(n)
    x.append(len(B_mejor))
    y_mejor.append(merge_sort_graph(B_mejor))

plt.plot(x, y_mejor)
plt.grid(True)
plt.show()

UnboundLocalError: cannot access local variable 'merge_times' where it is not associated with a value